In [2]:
!pip install -U bmaclient

Requirement already up-to-date: bmaclient in /home/lenovo/.local/lib/python3.8/site-packages (0.14.0)


In [3]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bmaclient import MonitoringAPI
from bmaclient.utils import get_api_key
from matplotlib.ticker import FormatStrFormatter, MaxNLocator

import datetime as dt
import scipy
from scipy import signal
import statsmodels.api as sm

In [ ]:
# BASE_DIR = os.path.dirname(os.path.abspath(__file__))
# client = MonitoringAPI(api_key='1JPZxKW5.RCpwvwK5O4T5hYFTzfPpSp9o2PdTtRwa')



def gps_data(start, end, station):
    client = MonitoringAPI(api_key='1JPZxKW5.RCpwvwK5O4T5hYFTzfPpSp9o2PdTtRwa')
    results = client.fetch_gps_position(
    timestamp__gte=start,
    timestamp__lt=end,
    station =station,
    nolimit=True,
    aggregation='mean')
    return results


In [30]:
babadan = gps_data('2022-03-01','2022-03-10','babadan')
babadan = pd.DataFrame(babadan)
babadan

,timestamp,east,north,up,err_east,err_north,err_up,orbit
0,2022-03-01T13:55:00+07:00,434984.496092,9.168033e+06,1305.510718,0.026011,0.013917,0.037881,2
1,2022-03-01T23:55:00+07:00,434984.499950,9.168033e+06,1305.511393,0.003439,0.001861,0.005137,0
2,2022-03-02T13:55:00+07:00,434984.502813,9.168033e+06,1305.497349,0.027469,0.014596,0.039728,2
3,2022-03-02T23:55:00+07:00,434984.502173,9.168033e+06,1305.500658,0.003245,0.001875,0.004878,0
4,2022-03-03T13:55:00+07:00,434984.501210,9.168033e+06,1305.499448,0.029191,0.015966,0.043309,2
5,2022-03-03T23:55:00+07:00,434984.499757,9.168033e+06,1305.499642,0.003486,0.002016,0.005044,0
6,2022-03-04T13:55:00+07:00,434984.498857,9.168033e+06,1305.476387,0.028480,0.016158,0.040596,2
7,2022-03-04T23:55:00+07:00,434984.503729,9.168033e+06,1305.495884,0.003272,0.001886,0.004970,0
8,2022-03-05T13:55:00+07:00,434984.500602,9.168033e+06,1305.489510,0.027596,0.015487,0.038925,2
9,2022-03-05T23:55:00+07:00,434984.501869,9.168033e+06,1305.495755,0.003253,0.001847,0.004715,0


In [24]:
#babadan['east'] = np.radians(babadan['east'])
#babadan['north'] = np.radians(babadan['north'])

#Change grawah timestamp data type to numerical value
babadan['timestamp'] = pd.to_datetime(babadan['timestamp'])
babadan['timestamp']=babadan['timestamp'].map(dt.datetime.toordinal)
#Grawah median filter
babadan['east']= scipy.signal.medfilt(babadan['east'],kernel_size=3)
babadan['north']= scipy.signal.medfilt(babadan['north'],kernel_size=3)

#Grawah linear Regression
X = sm.add_constant(babadan['timestamp'])
modelgx = sm.OLS(babadan['east'],X)
resultsgx = modelgx.fit()
y_predictgx = resultsgx.params[0] + resultsgx.params[1]*babadan['east']

X = sm.add_constant(babadan['timestamp'])
modelgy = sm.OLS(babadan['north'],X)
resultsgy = modelgy.fit()
y_predictgy = resultsgy.params[0] + resultsgy.params[1]*babadan['timestamp']

In [27]:
resultsgy.params[1]

-0.0005526163149625063

In [29]:
print(resultsgy.params[1], resultsgx.params[1])

-0.0005526163149625063 0.00012556667206808925
